# WEEK 3

# Basic Models

* **encoder-decoder architecture**: An encoder embeds a given input into a latent space which is fed to an RNN to "decode" it into a sequence. Examples:
  1. Sequence-to-sequence: automatic translation. Feed input sequence (french) into an encoder RNN, and then feed the latest hidden state as input to the decoder. The decoder sends its output at $t$ to the $t+1$ input to generate the translated sentence.
  2. Image captioning: explaining what's in an image. Similar to 1, but the hidden state is achieved by feeding an image to an RNN and removing, f.e., the last classifier network so we have the highest embedded representation as input to the decoder.

# Picking the Most Likely Sentence

Instead of sampling as we did before with the "language" mdoel, here we are interested in the best possible outcome. The formalization is as follows:

### Language model:

Before we sampled from a trained RNN to obtain feasible sentences by inputting a zero vector and then sampling from the RNN's  
$P(y^{<1>}, ..., y^{<T_y>})$

Now we have an extra condition: we don't want random feasible sentences but the best translation for our input sentence:  
$P(y^{<1>}, ..., y^{<T_y>} | x^{<1>}, ..., x^{<T_x>})$  
Which can be seen as a **conditional language model**

The optimization objective is then:

\begin{equation}
\begin{aligned}
argmax &= P(y^{<1>}, ..., y^{<T_y>} | x)\\
y^{<1>}, ..., y^{<T_y>} &
\end{aligned}
\end{equation}

In other words, "pick the $y$ sequence of english words that maximizes the likelihood given the $x$ sequence of french words. Note that there is NOT independence among the words in a sentence:
$P(y^{<1>}, ..., y^{<T_y>}) \neq \prod_{t=1}^{T_y} P(y^{<t>})$

And therefore a greedy search picking the best candidate at each stage will not necessarily provide the best joint distribution. Also note that checking all the combinations will require $|V|^{T_y}$ computations which is in most cases unfeasible.

# Beam Search:

Instead of searching the  $|V|^{T_y}$ space, set the hyperparameter $B$ as the total number of combinations to be stored. Then greedily increase $t$: for each of our current $B$-best combinations, check through the whole vocabulary for the best next word (by sending the combination so far plus the extra word through the RNN), and then collect the $b$-best outcomes, which will replace the existing ones.

Therefore we have $T_y*B*|V|$ computations, and for low B we already retain good chances of finding a close-to-optimal solution.

Formalization remember:

$P(y^{<1>}, y^{<2>} | x) = P(y^{<1>} | x) P(y^{<2>} | x, y^{<1>})$

Note that for $B=1$, we have the naive approach discussed before, where uncorrelated words are assumed.


# Refinements to Beam Search:

## Log normalization:

As in many similar problems, for numerical stability we reformulate the following:

\begin{equation}
\begin{aligned}
arg&max \prod_{t=1}^{T_y} P(y^{<t>} | x, y^{<1>}, ..., y^{<t-1>})\\
y &
\end{aligned}
\end{equation}

As the following with identical outcome

\begin{equation}
\begin{aligned}
arg&max \sum_{t=1}^{T_y} log P(y^{<t>} | x, y^{<1>}, ..., y^{<t-1>})\\
y &
\end{aligned}
\end{equation}

## Length normalization:
    
We also observe that this system naturally penalizes longer sequences, as they either have same or worse score than shorter ones. Introducing a normalizing factor that depends on the length helps overcome this issue:

\begin{equation}
\begin{aligned}
\frac{1}{T_y^\alpha} \sum_{t=1}^{T_y} log P(y^{<t>} | x, y^{<1>}, ..., y^{<t-1>})\\
\end{aligned}
\end{equation}

The $\alpha$ soft variable sets a compromise between no normalization at all for $\alpha=0$ and normalization proportional to the sentence length for $\alpha=1$. This is a hyperparameter to tune. Empirically it has been shown that values around $\alpha=0.7$ show good results.

The process is then to run the entire beam search up to a fixed length, and keep a record of the B-best sentences at each step. Finally normalize them against this formula depending on their length and pick the best one.

Note that a better B is always positive in terms of quality, but it increases the computational costs, and the gain of increasing B decreases greatly with the size of b. For production, 3, 10 or even 100 can be seen. For research, where the models are squeezed, 1000 or even 3000 can be seen.

Note that unlike BFS and other search algos this doesnt guarantee to find the global optimum.

# Error Analysis in Beam Search:

* The task of beam search is to maximize the joint probabilities as returned by the RNN decoder. If the pipeline is not performing well, it can be the RNN's or the BSs fault. Increasing B if it is RNN's fault is basically a waste of time.

* To find out, we need a curated translation. Send it through the RNN, and then send the translation accepted by beam search. If, for this mispredicted translation, the probability of the curated one is bigger, we know that beam search messed up. If OTOH the RNN outputs bigger probability for the mistranslated than for the curated, we know that the RNN has to be improved.

* Do this for many mistranslated examples, and if beam search is failing in many, it may be a good idea to increase B

# Bleu Score

*Papineni et al 2002: A method for automatic evaluation of machine translation*

"Bilingual Evaluation Understudy": understudy is an agent (person) that can take over when the tenure is away.

* Problematic: several sentences could be equally good/close to optimum. This metric is far from perfect but had and still has a deep impact in the NLP community.

Setup: we have a single translation but several targets:
* Target 1: The cat is on the mat
* Target 2: There is a cat on the mat
* Prediction: The cat the cat on the mat.

Idea: given any set of $n$ subsequent words (n-grams: unigrams, bigrams, trigrams etc...) in the predicted phrase,
1. make a histogram of the unique n-grams (i.e. repeated n-grams have one key but higher value): `hist = {k: v+1 for k in n_grams}`
2. Clip each count to the maximum number of times that the n-gram appears in either target `clipped = {k:max(count(find(k, target1), count(find(k, target2)) etc) for k,v in hist.items()}`. For example, "the cat" appears 2 times in the hist, but only 1 per sentence so the 2 is clipped down to 1
3. Divide the sum of the clipped values by the sum of the hist values

$
p_n = \frac{\sum_{n-grams \in \hat y} clip_{n-gram}}{\sum_{n-grams \in \hat y} hist_{n-gram}}
$

If the prediction is identical to any target, the score will be one. This helps as it brings a single, scalar score for this problem.


## Combined bleu score:

Usually, the score is taken for several sizes and then is collapsed as follows:

$
BP * exp(average([p_1, p_2, p_3, ...]))
$

Where BP is a brevity penalty, since shorter sentences will more likely be correct than longer ones. BP is 1 if the output length is bigger than some threshold, and otherwise

$
exp(1- \frac{threshold}{reference})
$

Where reference is some other threshold. Basically an exponential decay model.

# Attention Model Intuition

When the length of the sequence increases, the bleu score decreases (even with brevity penalty): this reflects how humans also translate: they dont memorize all at once and then translate, but go by smaller pieces.

The attention model (Bahdanau et al 2014 *Neural machine translation by jointly learning to align and translate*) allows to keep top score for arbitrarily long sequences.

## Setup:

* Lets pick a BRNN (more often LSTM than GRU, and maybe more than 1 layer) and feed the input text to it, to generate an encoding at each step of the sequence.

* Now each encoding is enriched by linearly combining it with the surrounding encodings to create a compromise between a local and rich representation: if we just picked the hidden state after a very long sentence, likely many of the details are lost on the way.

* The linear combination (weighted sum) is regulated by a set of $\alpha$ weights that determine the "attention span" at a given point. Ideally, the weights allow all the needed encodings for a short-term translation to pass through, and no more. Since each point relates to all other points, there is potentially a quadratic number of weights to be learned on the top of the BRNN.

* On the top of the model is the decoder RNN. Instead of performing a vanilla beam search given the encoded summary as an initial step, now the linearly combined encodings for each step are added, step by step, together with the previous predictions.

This is an intuition only, more details in the next video

# Attention Model

More details:

* The set of $\alpha$ weights for a given enriched encoding are all non-negative and add up to 1. Note that the encoding at every different input word has a different set of $\alpha$ weights that have to be determined (quadratic runtime, but ok for relatively short sentences)

* The enriched encoding is then a weighted sum $\sum \alpha_i concat(a_{forward}, a_{backward})$ between each alpha and the concatenation of the forward and backward hidden $a$ state of the BRNN.

Note that we still don't know how to determine the $\alpha$ weights or how does exactly the decoder receive its last hidden state and new enriched encoding as input. This is both addressed as follows:

1. First, concatenate the last hidden state of the decoder with the current non-enriched output of the encoder, and send them through a simple NN. It has to be small since it is computed many times. The output of that NN will have same dimensionality as the $\alpha$ vector of weights, and will basically determine the attention to the context: "It seems pretty natural that it depends on those 2 inputs, but we don't know the function so training a NN seems a fair approach".

2. Send the output of the NN through a softmax function: The result is exactly the definition for the $\alpha$ weights at the current point.

3. So the current enriched encoding is the weighted sum between the current $\alpha$ vector and the corresponding encodings. This is fed to the decoder, concatenated with the last predicted word.

4. The decoder will update its hidden state and process the concatenated input, outputing a further predicted word. The whole process is carried out until the decoder outputs EOS.

## Visualizations of attention

The attention weights can be plotted as a matrix, which gives a visual of how does the attention span evolve through the sentence.

## Other applications:

This has also been succesfully applied for image captioning (Xu et al 2015 *Neural image caption generation with visual attention*)

# Speech Recognition

* Discussion about preprocessing: STFT, phoneme model...
* Data size: In academia 300, 3000h are not uncommon, but top commercial systems take 100k to 1 million hours.
* Discusion about resolution: audio sequences have way bigger frequency than token-based sequences

* Attention model would work here

## CTC cost for speech recognition

(Connectionist temporal classification, Graves et al 2006).

* The idea is that a symbol can be identified in many repeated steps, and between symbols they may be unknown 'blank' assigned characters (which are different from 'space' characters), so **repeated symbols not separated by a blank are collapsed into one**:  `__c_oo_o_kk___b_ooooo__oo__kkk -> cookbook`

Check the paper

# Trigger Word Detection

We know that much that this can be explained in one slide: This is like entity recognition, and our entity is a trigger word: for each input (which is some feature such as an STFT frame) we predict whether we just finished a trigger word (1) or not (0).

To balance data, instead of a single 1 a row of several 1s can be set. This is a hack but works

# Assignments